In [ ]:
# Install the OpenSlide C library and Python bindings
# After installing these libraries, use `Runtime -> restart and run all` on the menu
!apt-get install openslide-tools
!pip install openslide-python

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from openslide import open_slide, __library_version__ as openslide_version
import os
from PIL import Image
from skimage.color import rgb2gray
import random
import pandas as pd
import tensorflow as tf
from sklearn.metrics import classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def make_dir(base_path, level):
    level_dir = f'{base_path}level{level}'
    if not os.path.exists(level_dir):
        os.mkdir(level_dir)
    train_image_dir = os.path.join(level_dir,'train_image/')
    train_mask_dir = os.path.join(level_dir,'train_mask/')

    val_image_dir = os.path.join(level_dir,'val_image/')
    val_mask_dir = os.path.join(level_dir,'val_mask/')

    test_image_dir = os.path.join(level_dir,'test_image/')
    test_mask_dir = os.path.join(level_dir,'test_mask/')

    if not os.path.exists(train_image_dir):
        os.mkdir(train_image_dir)
    if not os.path.exists(train_mask_dir):
        os.mkdir(train_mask_dir)
    if not os.path.exists(val_image_dir):
        os.mkdir(val_image_dir)
    if not os.path.exists(val_mask_dir):
        os.mkdir(val_mask_dir)    
    if not os.path.exists(test_image_dir):
        os.mkdir(test_image_dir)
    if not os.path.exists(test_mask_dir):
        os.mkdir(test_mask_dir)
    return(train_image_dir, train_mask_dir, val_image_dir, val_mask_dir, test_image_dir, test_mask_dir)

In [ ]:
base_path = '/content/'
train_image_dir5, train_mask_dir5, val_image_dir5, val_mask_dir5, test_image_dir5, test_mask_dir5 = make_dir(base_path, level = 5)
train_image_dir4, train_mask_dir4, val_image_dir4, val_mask_dir4, test_image_dir4, test_mask_dir4 = make_dir(base_path, level = 4)
train_image_dir3, train_mask_dir3, val_image_dir3, val_mask_dir3, test_image_dir3, test_mask_dir3 = make_dir(base_path, level = 3)

In [ ]:
# create a folder called 'Applied Deep Learning' in the Google Drive

!unzip -q '/content/drive/MyDrive/Applied Deep Learning/project/level4/train_mask.zip' -d /content/level4/train_mask/
!unzip -q '/content/drive/MyDrive/Applied Deep Learning/project/level4/train_image.zip' -d /content/level4/train_image/
!unzip -q '/content/drive/MyDrive/Applied Deep Learning/project/level4/val_mask.zip' -d /content/level4/val_mask/
!unzip -q '/content/drive/MyDrive/Applied Deep Learning/project/level4/val_image.zip' -d /content/level4/val_image/
!unzip -q '/content/drive/MyDrive/Applied Deep Learning/project/level4/test_mask.zip' -d /content/level4/test_mask/
!unzip -q '/content/drive/MyDrive/Applied Deep Learning/project/level4/test_image.zip' -d /content/level4/test_image/

!unzip -q '/content/drive/MyDrive/Applied Deep Learning/project/level5/train_mask.zip' -d /content/level5/train_mask/
!unzip -q '/content/drive/MyDrive/Applied Deep Learning/project/level5/train_image.zip' -d /content/level5/train_image/
!unzip -q '/content/drive/MyDrive/Applied Deep Learning/project/level5/val_mask.zip' -d /content/level5/val_mask/
!unzip -q '/content/drive/MyDrive/Applied Deep Learning/project/level5/val_image.zip' -d /content/level5/val_image/
!unzip -q '/content/drive/MyDrive/Applied Deep Learning/project/level5/test_mask.zip' -d /content/level5/test_mask/
!unzip -q '/content/drive/MyDrive/Applied Deep Learning/project/level5/test_image.zip' -d /content/level5/test_image/

!unzip -q '/content/drive/MyDrive/Applied Deep Learning/project/level3/train_mask.zip' -d /content/level3/train_mask/
!unzip -q '/content/drive/MyDrive/Applied Deep Learning/project/level3/train_image.zip' -d /content/level3/train_image/
!unzip -q '/content/drive/MyDrive/Applied Deep Learning/project/level3/val_mask.zip' -d /content/level3/val_mask/
!unzip -q '/content/drive/MyDrive/Applied Deep Learning/project/level3/val_image.zip' -d /content/level3/val_image/
!unzip -q '/content/drive/MyDrive/Applied Deep Learning/project/level3/test_mask.zip' -d /content/level3/test_mask/
!unzip -q '/content/drive/MyDrive/Applied Deep Learning/project/level3/test_image.zip' -d /content/level3/test_image/

In [ ]:
!cp '/content/drive/MyDrive/Applied Deep Learning/project/level4/level4_train_data.txt' /content/
!cp '/content/drive/MyDrive/Applied Deep Learning/project/level4/level4_val_data.txt' /content/
!cp '/content/drive/MyDrive/Applied Deep Learning/project/level4/level4_test_data.txt' /content/

!cp '/content/drive/MyDrive/Applied Deep Learning/project/level5/level5_train_data.txt' /content/
!cp '/content/drive/MyDrive/Applied Deep Learning/project/level5/level5_val_data.txt' /content/
!cp '/content/drive/MyDrive/Applied Deep Learning/project/level5/level5_test_data.txt' /content/

!cp '/content/drive/MyDrive/Applied Deep Learning/project/level3/level3_train_data.txt' /content/
!cp '/content/drive/MyDrive/Applied Deep Learning/project/level3/level3_val_data.txt' /content/
!cp '/content/drive/MyDrive/Applied Deep Learning/project/level3/level3_test_data.txt' /content/

In [ ]:
IMG_SIZE = 299
batch_size = 32
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# data generator
def data_generator(train_image_dir, val_image_dir, test_image_dir):
    train_datagen = ImageDataGenerator(rescale = 1./255)
    test_datagen = ImageDataGenerator(rescale = 1./255)
    # train set
    train_generator = train_datagen.flow_from_directory(
        train_image_dir,
        target_size = (IMG_SIZE, IMG_SIZE),
        batch_size = batch_size,
        class_mode = 'binary',
        shuffle=True
    )
    validation_generator = test_datagen.flow_from_directory(
        val_image_dir, 
        target_size = (IMG_SIZE, IMG_SIZE),
        batch_size = batch_size,
        class_mode = 'binary',
        shuffle=True
    )

    for data_batch, labels_batch in train_generator:
        print('Train data: ')
        print('data batch shape:', data_batch.shape)
        print('labels batch shape:', labels_batch.shape)
        break

    for data_batch, labels_batch in validation_generator:
        print('\n')
        print('Validation data: ')
        print('data batch shape:', data_batch.shape)
        print('labels batch shape:', labels_batch.shape)
        break
    # test set
    test_generator = test_datagen.flow_from_directory(
        test_image_dir,
        target_size = (IMG_SIZE, IMG_SIZE),
        batch_size = batch_size,
        class_mode = 'binary',
        shuffle = True
    )
    return(train_generator, validation_generator, test_generator)

# data generator with augmentation
def data_generator_aug(train_image_dir, val_image_dir, test_image_dir):
    # train set
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=90,
        # width_shift_range=0.2,
        # height_shift_range=0.2,
        # shear_range=0.2,
        # zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2)
    test_datagen = ImageDataGenerator(rescale = 1./255)
    # train set
    train_generator = train_datagen.flow_from_directory(
        train_image_dir,
        target_size = (IMG_SIZE, IMG_SIZE),
        batch_size = batch_size,
        class_mode = 'binary',
        shuffle=True
    )
    validation_generator = test_datagen.flow_from_directory(
        val_image_dir, 
        target_size = (IMG_SIZE, IMG_SIZE),
        batch_size = batch_size,
        class_mode = 'binary',
        shuffle=True
    )

    for data_batch, labels_batch in train_generator:
        print('Train data: ')
        print('data batch shape:', data_batch.shape)
        print('labels batch shape:', labels_batch.shape)
        break

    for data_batch, labels_batch in validation_generator:
        print('\n')
        print('Validation data: ')
        print('data batch shape:', data_batch.shape)
        print('labels batch shape:', labels_batch.shape)
        break
    # test set
    test_generator = test_datagen.flow_from_directory(
        test_image_dir,
        target_size = (IMG_SIZE, IMG_SIZE),
        batch_size = batch_size,
        class_mode = 'binary',
        shuffle = True
    )
    return(train_generator, validation_generator, test_generator)

# Multi-level Model

In [ ]:
print('level 4')
level4_train_generator, level4_validation_generator, level4_test_generator = data_generator_aug(train_image_dir4, 
                                                                                                val_image_dir4, 
                                                                                                test_image_dir4)
print()
print('level 3')
level3_train_generator, level3_validation_generator, level3_test_generator = data_generator_aug(train_image_dir3, 
                                                                                                val_image_dir3, 
                                                                                                test_image_dir3)

In [ ]:
# def combine_generator(gen1, gen2):
#     while True:
#         yield(next(gen1), next(gen2))

In [ ]:
# multi_train_generator = combine_generator(level3_train_generator, level4_train_generator)
# multi_val_generator = combine_generator(level3_validation_generator, level4_validation_generator)
# multi_test_generator = combine_generator(level3_test_generator, level4_test_generator)

In [ ]:
IMG_SIZE = 299
vgg_base_1 = tf.keras.applications.VGG16(
            include_top = False,
            weights = 'imagenet',
            input_shape = (IMG_SIZE, IMG_SIZE, 3),
            pooling = None,
            classes = 2,
            classifier_activation = 'softmax',
        )
vgg_base_2 = tf.keras.applications.VGG16(
            include_top = False,
            weights = 'imagenet',
            input_shape = (IMG_SIZE, IMG_SIZE, 3),
            pooling = None,
            classes = 2,
            classifier_activation = 'softmax',
        )
vgg_base_1.trainable = False
vgg_base_2.trainable = False


high_res_level_input = tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
low_res_level_input = tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

# high resolution level
high_res_vgg = tf.keras.Sequential([
    vgg_base_1,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu')
])
high_res_vgg_output = high_res_vgg(high_res_level_input)

# low resolution level
low_res_vgg = tf.keras.Sequential([
    vgg_base_2,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu')
])
low_res_vgg_output = low_res_vgg(low_res_level_input)

# concatenate two output layers as input
concatenated_input = tf.keras.layers.concatenate([high_res_vgg_output, low_res_vgg_output])
flattened = tf.keras.layers.Flatten()(concatenated_input)

output = tf.keras.layers.Dense(128, activation="relu")(flattened)
output = tf.keras.layers.Dropout(0.4)(output)
output = tf.keras.layers.Dense(2, activation="softmax")(output)

vgg_multi_resolution_model = tf.keras.Model(inputs=[high_res_level_input, low_res_level_input], outputs=output)
vgg_multi_resolution_model.summary()

vgg_multi_resolution_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
                                   loss = 'sparse_categorical_crossentropy',
                                   metrics = ['accuracy'])

In [ ]:
tf.keras.utils.plot_model(vgg_multi_resolution_model)

In [ ]:
if not os.path.exists('/content/checkpoint'):
  os.mkdir('/content/checkpoint')
if not os.path.exists('/content/checkpoint/VGG'):
  os.mkdir('/content/checkpoint/VGG')
multi_vgg_checkpoint_path = '/content/checkpoint/VGG/multi_level_weights-improvement-{epoch:02d}-{val_accuracy:.2f}-{val_loss:.2f}.hdf5'
vgg_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(multi_vgg_checkpoint_path, 
                                                                    monitor='val_loss', 
                                                                    save_best_only=True, 
                                                                    mode='auto')

vgg_early_stop_callback = tf.keras.callbacks.EarlyStopping(
                              monitor='val_loss', min_delta=0, patience=10)

In [ ]:
epochs = 10
multi_level_vgg_history = vgg_multi_resolution_model.fit(multi_train_generator, 
                                                          epochs=epochs, 
                                                          validation_data=multi_val_generator,
                                                          callbacks=[vgg_checkpoint_callback, 
                                                                    vgg_early_stop_callback])